In [ ]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "referer": "https://www.card-gorilla.com/"
}

def get_super_detailed_info(card_idx):
    """HTML 태그가 섞인 초정밀 혜택(실적별 한도 등)을 텍스트로 추출"""
    url = f"https://api.card-gorilla.com:8080/v1/cards/{card_idx}"
    try:
        res = requests.get(url, headers=headers)
        card_data = res.json()
        
        # 'key_benefit' 섹션 접근
        key_benefits = card_data.get('key_benefit', [])
        all_details = []
        
        for kb in key_benefits:
            # 실적별 한도와 유의사항이 포함된 'info' 필드 추출
            html_content = kb.get('info', '')
            if html_content:
                # BeautifulSoup으로 깨끗한 텍스트로 변환
                soup = BeautifulSoup(html_content, 'html.parser')
                clean_text = soup.get_text(separator=" ", strip=True)
                all_details.append(f"[{kb.get('title')}]: {clean_text}")
        
        return " \n ".join(all_details)
    except:
        return "정보 수집 실패"

# --- 데이터 수집 루프 시작 ---
card_info_list = []
page = 1

print("🚀 전체 카드 데이터 수집을 시작합니다. (상세 내용 포함)")

while True:
    # 1. 현재 페이지의 카드 리스트 가져오기
    list_url = f"https://api.card-gorilla.com:8080/v1/cards?p={page}&perPage=20&cate=CRD"
    response = requests.get(list_url, headers=headers)
    data = response.json()
    goods_list = data.get("data", [])

    # 2. 만약 더 이상 수집할 데이터가 없으면 루프 종료
    if not goods_list:
        print(f"\n✅ 수집 완료! 총 {len(card_info_list)}개의 카드를 수집했습니다.")
        break

    print(f"\n📦 현재 {page}페이지 수집 중... (누적: {len(card_info_list)}건)")

    # 3. 리스트에 있는 개별 카드의 상세 정보 수집
    for card in goods_list:
        card_idx = card.get("idx")
        card_name = card.get("name")
        
        # 초정밀 정보 가져오기 루틴
        super_detail = get_super_detailed_info(card_idx)
        
        row = {
            "카드ID": card_idx,
            "카드명": card_name,
            "카드사": card.get("corp_txt"),
            "전월실적기준": card.get("pre_month_money"),
            "연회비": card.get("annual_fee_basic"),
            "초정밀혜택": super_detail
        }
        card_info_list.append(row)
        
        # 서버 매너를 위해 0.5초 대기 (차단 방지)
        time.sleep(0.5)

    page += 1 # 다음 페이지로 이동

# --- 결과 저장 ---
df_final = pd.DataFrame(card_info_list)
# 엑셀 깨짐 방지를 위해 utf-8-sig로 저장
df_final.to_csv("card_ultra_details_all.csv", index=False, encoding="utf-8-sig")

print("\n✨ 모든 데이터가 'card_ultra_details_all.csv'로 저장되었습니다.")

In [ ]:
import requests
import pandas as pd
import time
import re
from bs4 import BeautifulSoup

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "referer": "https://www.card-gorilla.com/"
}

def get_long_format_benefits(card_idx, base_info):
    """한 카드의 혜택들을 여러 개의 행(list of dicts)으로 분리하여 반환"""
    url = f"https://api.card-gorilla.com:8080/v1/cards/{card_idx}"
    rows = []
    
    try:
        res = requests.get(url, headers=headers)
        card_data = res.json()
        key_benefits = card_data.get('key_benefit', [])
        
        for kb in key_benefits:
            title = kb.get('title') # 카테고리 명 (예: 공과금)
            html_content = kb.get('info', '')
            
            if html_content and title:
                soup = BeautifulSoup(html_content, 'html.parser')
                text = soup.get_text(separator="\n", strip=True)
                
                # 🔍 실적/한도 공통 섹션 분리 로직
                split_keywords = ['할인한도안내', '적립한도안내', '유의사항']
                parts = re.split('|'.join(split_keywords), text, 1)
                
                benefit_desc = parts[0].strip() # 순수 혜택
                condition_text = text[len(parts[0]):].strip() if len(parts) > 1 else "" # 공통 실적/한도
                
                # 각 혜택별로 새로운 행 데이터 생성
                new_row = base_info.copy()
                new_row.update({
                    "혜택카테고리": title,
                    "순수혜택내용": benefit_desc,
                    "상세실적및한도": condition_text
                })
                rows.append(new_row)
                
        return rows
    except:
        return []

# --- 데이터 수집 실행부 ---
final_long_data = []
target_count = 20 # 테스트용 20개
collected_cards = 0
page = 1

print(f"🚀 RAG 최적화를 위해 데이터를 '행 구조'로 수집합니다... (목표: {target_count}개 카드)")



while collected_cards < target_count:
    list_url = f"https://api.card-gorilla.com:8080/v1/cards?p={page}&perPage=20&cate=CRD"
    response = requests.get(list_url, headers=headers)
    goods_list = response.json().get("data", [])

    if not goods_list: break

    for card in goods_list:
        if collected_cards >= target_count: break
            
        card_idx = card.get("idx")
        card_name = card.get("name")
        print(f"📡 [{collected_cards + 1}] {card_name} 혜택 분해 중...")
        
        # 기본 정보 구성
        base_info = {
            "카드ID": card_idx,
            "카드명": card_name,
            "카드사": card.get("corp_txt"),
            "전월실적기준(숫자)": card.get("pre_month_money"),
            "연회비": card.get("annual_fee_basic")
        }
        
        # 카테고리별로 행 분리 호출
        card_rows = get_long_format_benefits(card_idx, base_info)
        final_long_data.extend(card_rows)
        
        collected_cards += 1
        time.sleep(0.5)

    page += 1

# --- 결과 저장 ---
df_long = pd.DataFrame(final_long_data)
df_long.to_csv("card_rag_long_format.csv", index=False, encoding="utf-8-sig")

print(f"\n✨ 수집 완료! 'card_rag_long_format.csv' 파일을 확인해 보세요.")
print(f"총 생성된 데이터 행수: {len(df_long)}개")

In [ ]:
import requests
import json
import time
import re
from bs4 import BeautifulSoup

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "referer": "https://www.card-gorilla.com/"
}

def get_long_format_benefits(card_idx, base_info):
    """한 카드의 상세 혜택들을 개별 객체 리스트로 분리"""
    url = f"https://api.card-gorilla.com:8080/v1/cards/{card_idx}"
    rows = []
    try:
        res = requests.get(url, headers=headers)
        card_data = res.json()
        key_benefits = card_data.get('key_benefit', [])
        
        for kb in key_benefits:
            title = kb.get('title')
            html_content = kb.get('info', '')
            
            if html_content and title:
                soup = BeautifulSoup(html_content, 'html.parser')
                # 텍스트 정제 및 실적 조건 분리 로직
                text = soup.get_text(separator="\n", strip=True)
                
                split_keywords = ['할인한도안내', '적립한도안내', '유의사항']
                parts = re.split('|'.join(split_keywords), text, 1)
                
                benefit_desc = parts[0].strip()
                condition_text = text[len(parts[0]):].strip() if len(parts) > 1 else ""
                
                # RAG 입력용 최종 데이터 구조 (JSON의 한 항목이 됨)
                new_entry = base_info.copy()
                new_entry.update({
                    "혜택카테고리": title,
                    "순수혜택내용": benefit_desc,
                    "상세실적및한도": condition_text
                })
                rows.append(new_entry)
        return rows
    except:
        return []

# --- 전체 수집 실행부 ---
all_card_data = []
categories = ["CRD", "CHK"] # 신용카드와 체크카드 전체

print("🚀 전 종목 카드 데이터 수집 및 JSON 변환을 시작합니다...")

for cate in categories:
    page = 1
    cate_label = "신용카드" if cate == "CRD" else "체크카드"
    
    while True:
        list_url = f"https://api.card-gorilla.com:8080/v1/cards?p={page}&perPage=20&cate={cate}"
        response = requests.get(list_url, headers=headers)
        goods_list = response.json().get("data", [])

        if not goods_list: # 더 이상 데이터가 없으면 다음 카테고리로
            break

        print(f"📦 {cate_label} {page}페이지 분석 중... (현재 누적 데이터: {len(all_card_data)}건)")

        for card in goods_list:
            card_idx = card.get("idx")
            base_info = {
                "카드ID": card_idx,
                "카드명": card.get("name"),
                "카드사": card.get("corp_txt"),
                "카드종류": cate_label,
                "전월실적기준": card.get("pre_month_money"),
                "연회비": card.get("annual_fee_basic")
            }
            
            # 상세 혜택을 행(Object) 단위로 쪼개서 추가
            benefit_entries = get_long_format_benefits(card_idx, base_info)
            all_card_data.extend(benefit_entries)
            
            # 서버 부하 방지 매너 타임
            time.sleep(0.4)

        page += 1

# --- JSON 파일로 저장 ---
with open("card_data_all_long.json", "w", encoding="utf-8") as f:
    # 한글 깨짐 방지를 위해 ensure_ascii=False 사용
    json.dump(all_card_data, f, ensure_ascii=False, indent=4)

print(f"\n✨ 수집 완료! 총 {len(all_card_data)}개의 혜택 객체가 'card_data_all_long.json'에 저장되었습니다.")

In [ ]:
from dotenv import load_dotenv 

load_dotenv()

In [ ]:
import json
import random
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 1. 모델 설정 (프로젝트 가이드 준수)
llm = ChatOpenAI(model="gpt-5-nano", temperature=0.8)

def generate_sample_questions(input_file):
    # 2. 파일 로드
    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            cards = json.load(f)
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {input_file}")
        return

    dataset = []

    # 3. 질문 생성을 위한 프롬프트
    prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 카드 추천 챗봇의 학습 데이터를 생성하는 전문가입니다."),
        ("user", """아래 제공된 [카드 상세 정보]를 바탕으로, 실제 사용자가 카드 추천을 받기 위해 물어볼 법한 자연스러운 질문 1개를 생성하세요.

        [카드 상세 정보]: {card_info}

        조건:
        1. 질문(anchor)은 "~살이고 ~한 상황인데 카드 추천해줘"와 같이 구체적인 페르소나가 느껴지는 구어체여야 합니다.
        2. 질문은 반드시 제공된 카드 혜택 중 하나와 강력하게 연결되어야 합니다.
        3. 결과는 반드시 아래 JSON 형식으로만 출력하세요.
        {{
            "anchor": "생성된 질문",
            "positive": "질문에 딱 맞는 구체적인 혜택 내용 추출"
        }}
        """)
    ])

    # 4. 상위 3개 카드에 대해 질문 생성
    target_cards = cards[:3]  # 테스트를 위해 3개만 진행
    
    print(f"총 {len(target_cards)}건의 샘플 질문 생성을 시작합니다...")

    for i, card in enumerate(target_cards):
        # AI에게 넘겨줄 텍스트 데이터 준비 (데이터 구조가 달라도 대응 가능하도록 통째로 문자열화)
        card_info_str = json.dumps(card, ensure_ascii=False)
        
        chain = prompt | llm
        try:
            response = chain.invoke({"card_info": card_info_str})
            # JSON 응답만 깔끔하게 추출
            content = response.content.strip().replace('```json', '').replace('```', '')
            ai_data = json.loads(content)
            
            # Negative 데이터 구성 (해당 카드의 다른 부분이나 일반적인 멘트)
            # 데이터 구조를 정확히 모를 때를 대비해 범용적인 문구를 넣었습니다.
            negative_text = "이 카드는 해당 질문과 직접적인 관련이 없는 연회비 또는 전월 실적 안내 정보를 포함하고 있습니다."

            result_item = {
                "anchor": ai_data['anchor'],
                "positive": ai_data['positive'],
                "negative": negative_text,
                "metadata": {
                    "card_name": card.get('card_name', 'Unknown Card'),
                    "card_id": i + 1
                }
            }
            dataset.append(result_item)
            print(f"[{i+1}/3] 생성 완료")
            
        except Exception as e:
            print(f"에러 발생: {e}")

    # 5. 결과 출력 및 저장
    print("\n--- 생성된 데이터셋 샘플 ---")
    print(json.dumps(dataset, ensure_ascii=False, indent=2))
    
    with open('generated_qa.json', 'w', encoding='utf-8') as f:
        json.dump(dataset, f, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    generate_sample_questions('card_data_all_long.json')